In [12]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
# 👇 이 부분을 수정했습니다!
from langchain.output_parsers import CommaSeparatedListOutputParser
import os
from dotenv import load_dotenv

# .env 파일에서 환경 변수를 로드합니다.
load_dotenv()

try:
    # ChatOpenAI 모델 초기화
    llm = ChatOpenAI(
        api_key=os.getenv("GROQ_API_KEY"),
        base_url="https://api.groq.com/openai/v1",
        model="moonshotai/kimi-k2-instruct-0905",
        temperature=0.7
    )

    # 출력 파서 초기화
    output_parser = CommaSeparatedListOutputParser()

    # 프롬프트 템플릿 생성
    template = """
    당신은 한국 여행 전문가입니다. 사용자의 관심 분야와 관련된 한국의 유명한 장소나 활동 5가지를 추천해주세요.
    {format_instructions}

    관심 분야: {topic}
    """
    prompt = ChatPromptTemplate.from_template(
        template,
        partial_variables={"format_instructions": output_parser.get_format_instructions()}
    )

    # 체인 생성
    chain = prompt | llm | output_parser

    # 체인 실행
    topic = "음식"
    result = chain.invoke({"topic": topic})

    # 결과 출력
    print(f"관심 분야: \"{topic}\"")
    print("추천 리스트:", result)

except Exception as e:
    print(f"오류가 발생했습니다: {e}")

관심 분야: "음식"
추천 리스트: ['전주한옥마을 맛집투어', '서울 광장시장 족발골목', '부산 자갈치시장 활어회', '제주 흑돼지 구이', '전남 목포 삼희네 해장국']


In [10]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
# 👇 이 부분을 수정했습니다!
from langchain.output_parsers import EnumOutputParser
from enum import Enum
import os
from dotenv import load_dotenv

# .env 파일에서 환경 변수를 로드합니다.
load_dotenv()

# 감정 분류를 위한 Enum 클래스 정의
class Sentiment(str, Enum):
    POSITIVE = "긍정"
    NEGATIVE = "부정"
    NEUTRAL = "보통"

try:
    # ChatOpenAI 모델 초기화
    llm = ChatOpenAI(
        api_key=os.getenv("GROQ_API_KEY"),
        base_url="https://api.groq.com/openai/v1",
        model="moonshotai/kimi-k2-instruct-0905",
        temperature=0
    )

    # 출력 파서 초기화
    output_parser = EnumOutputParser(enum=Sentiment)

    # 프롬프트 템플릿 생성
    template = """
    당신은 영화 리뷰의 감정을 분석하는 전문가입니다. 주어진 리뷰의 감정을 "긍정", "부정", "보통" 중 하나로 분류해주세요.
    {format_instructions}

    영화 리뷰:
    {review}
    """
    prompt = ChatPromptTemplate.from_template(
        template,
        partial_variables={"format_instructions": output_parser.get_format_instructions()}
    )

    # 체인 생성
    chain = prompt | llm | output_parser

    # 테스트 리뷰
    reviews = [
        "이 영화 정말 재미없어요. 시간 낭비였습니다.",
        "배우들의 연기가 훌륭하고 스토리도 감동적이었어요!",
        "그냥 무난한 영화였습니다. 나쁘지도 좋지도 않아요."
    ]

    # 각 리뷰에 대해 체인 실행 및 결과 출력
    for review in reviews:
        result = chain.invoke({"review": review})
        print(f"리뷰: \"{review}\"")
        print(f"분석 결과: {result.value}\n")

except Exception as e:
    print(f"오류가 발생했습니다: {e}")

리뷰: "이 영화 정말 재미없어요. 시간 낭비였습니다."
분석 결과: 부정

리뷰: "배우들의 연기가 훌륭하고 스토리도 감동적이었어요!"
분석 결과: 긍정

리뷰: "그냥 무난한 영화였습니다. 나쁘지도 좋지도 않아요."
분석 결과: 보통



In [8]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List
import os
from dotenv import load_dotenv
#2-3
# .env 파일에서 환경 변수를 로드합니다.
load_dotenv()

# 추출할 정보에 대한 Pydantic 모델 정의
class StudentInfo(BaseModel):
    name: str = Field(description="학생의 이름")
    age: int = Field(description="학생의 나이")
    major: str = Field(description="학생의 전공")
    hobbies: List[str] = Field(description="학생의 취미 리스트")
    goal: str = Field(description="학생의 목표")


try:
    # ChatOpenAI 모델 초기화
    llm = ChatOpenAI(
    api_key=os.getenv("GROQ_API_KEY"), 
    base_url="https://api.groq.com/openai/v1",
    model="moonshotai/kimi-k2-instruct-0905", 
    temperature=0.7
)

    # 출력 파서 초기화
    output_parser = PydanticOutputParser(pydantic_object=StudentInfo)

    # 프롬프트 템플릿 생성
    template = """
    당신은 주어진 자기소개 텍스트에서 학생의 정보를 추출하는 전문가입니다.
    아래 형식에 맞게 정보를 추출해주세요.
    {format_instructions}

    자기소개:
    {introduction}
    """
    prompt = ChatPromptTemplate.from_template(
        template,
        partial_variables={"format_instructions": output_parser.get_format_instructions()}
    )

    # 체인 생성
    chain = prompt | llm | output_parser

    # 예시 자기소개
    introduction = "안녕하세요! 저는 김민수이고 22살입니다. 컴퓨터공학을 전공하고 있어요. 취미로는 게임하기, 영화보기, 코딩을 좋아합니다. 앞으로 훌륭한 개발자가 되는 것이 목표입니다."

    # 체인 실행
    result = chain.invoke({"introduction": introduction})

    # 결과 출력
    print(f"자기소개: \"{introduction}\"")
    print("\n추출된 정보:")
    print(result.model_dump_json(indent=4, ensure_ascii=False))

except Exception as e:
    print(f"오류가 발생했습니다: {e}")

자기소개: "안녕하세요! 저는 김민수이고 22살입니다. 컴퓨터공학을 전공하고 있어요. 취미로는 게임하기, 영화보기, 코딩을 좋아합니다. 앞으로 훌륭한 개발자가 되는 것이 목표입니다."

추출된 정보:
오류가 발생했습니다: BaseModel.model_dump_json() got an unexpected keyword argument 'ensure_ascii'


In [7]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
import os
from dotenv import load_dotenv

# .env 파일에서 환경 변수를 로드합니다.
load_dotenv()

try:
    # ChatOpenAI 모델 초기화
    llm = ChatOpenAI(
    api_key=os.getenv("GROQ_API_KEY"), 
    base_url="https://api.groq.com/openai/v1",
    model="moonshotai/kimi-k2-instruct-0905", 
    temperature=0.7
)

    # 추출할 정보의 스키마 정의
    response_schemas = [
        ResponseSchema(name="destination", description="여행지"),
        ResponseSchema(name="duration", description="여행 기간 (예: 2박 3일)"),
        ResponseSchema(name="budget", description="총 예산"),
        ResponseSchema(name="rating", description="추천도 (1-5점)"),
        ResponseSchema(name="activities", description="주요 활동 리스트 (콤마로 구분)")
    ]

    # 출력 파서 초기화
    output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

    # 프롬프트 템플릿 생성
    template = """
    당신은 여행 후기 텍스트에서 핵심 정보를 추출하는 시스템입니다.
    아래 형식에 맞게 정보를 추출해주세요.
    {format_instructions}

    여행 후기:
    {review}
    """
    prompt = ChatPromptTemplate.from_template(
        template,
        partial_variables={"format_instructions": output_parser.get_format_instructions()}
    )

    # 체인 생성
    chain = prompt | llm | output_parser

    # 예시 여행 후기
    review = "지난 주에 부산으로 2박 3일 여행을 다녀왔어요. 총 30만원 정도 썼는데 해운대에서 바다구경하고, 자갈치시장에서 회 먹고, 감천문화마을도 구경했어요. 정말 만족스러운 여행이었습니다. 5점 만점에 4점 정도 줄 수 있을 것 같아요."

    # 체인 실행
    result = chain.invoke({"review": review})

    # 결과 출력
    import json
    print(f"여행 후기: \"{review}\"\n")
    print("추출된 정보:")
    # 보기 좋게 json 형식으로 출력
    print(json.dumps(result, indent=4, ensure_ascii=False))


except Exception as e:
    print(f"오류가 발생했습니다: {e}")

여행 후기: "지난 주에 부산으로 2박 3일 여행을 다녀왔어요. 총 30만원 정도 썼는데 해운대에서 바다구경하고, 자갈치시장에서 회 먹고, 감천문화마을도 구경했어요. 정말 만족스러운 여행이었습니다. 5점 만점에 4점 정도 줄 수 있을 것 같아요."

추출된 정보:
{
    "destination": "부산",
    "duration": "2박 3일",
    "budget": "30만원",
    "rating": "4점",
    "activities": "해운대 바다구경, 자갈치시장 회 먹기, 감천문화마을 구경"
}
